In [1]:
import gensim
from xlrd import open_workbook
import os
from nltk.tokenize import word_tokenize, sent_tokenize
import json
import pandas as pd
from nltk.corpus import stopwords
import pprint
import nltk
import re
from os import listdir, makedirs
from os.path import isfile, join, exists
import os
from nltk.corpus import stopwords
from xlrd import open_workbook
import json
from gensim import similarities
import numpy as np

In [2]:
       
unspec_df = pd.read_excel('ignore/UNSPSC English v220601 project.xlsx','Sheet1', index_col=None, na_values=['NA'])
unspec = unspec_df.drop(['Family Definition','Family','Family Title','Segment Title','Segment Definition','Segment','Version','Key','Synonym','Acronym'],axis = 1)
cols = ['Class Title','Class Definition']
cols2 =['Commodity Title','Commodity Definition']
col3 = ['class','commodity']
unspec["class"] = unspec[cols].apply(lambda row: ','.join(row.values.astype(str)), axis=1)
unspec["commodity"] = unspec[cols2].apply(lambda row: ','.join(row.values.astype(str)), axis=1)
unspec["combine"]= unspec[col3].apply(lambda row: ', '.join(row.values.astype(str)),axis=1)
unspec["combine"]=[row.replace('nan','')for row in unspec["combine"]]
unspec2 = unspec.drop(['Class','Class Title','Class Definition','Commodity Title','Commodity Definition','class','commodity'],axis=1)
unspec2=unspec2.rename(columns={"Commodity":"level_1"})
nums = unspec2['level_1'].tolist()

In [3]:
county_df = pd.read_excel('ignore/eCAPS_COMM_11072019.xlsx','COMM_CLS', index_col=None, na_values=['NA'])
county_df2 = pd.read_excel('ignore/eCAPS_COMM_11072019.xlsx','COMM_ITM', index_col=None, na_values=['NA'])
county_df.drop(['COMM_GP','COMM_DET','ACT_INACT_FL','NAICS_CD'],axis=1)
county_df2.drop(['COMM_GP','COMM_DET','ACT_INACT_FL','NAICS_CD'],axis=1)
countyresult = pd.merge(county_df,county_df2,on='COMM_CLS')
cr = countyresult.drop(['DSCR_EXT_y','DSCR_EXT_DV_x','DSCR_EXT_x','COMM_DSCR_UP_x','COMM_CD_x','COMM_ITM_x','COMM_GP_x','COMM_DET_x','ACT_INACT_FL_x','NAICS_CD_x','COMM_DET_y','COMM_GP_y','COMM_DET_y','ACT_INACT_FL_y','NAICS_CD_y','COMM_DSCR_UP_y','DSCR_EXT_DV_y'],axis=1)
cr2 = cr.rename(columns={"KEYWD_x":"class","KEYWD_y":"commodity","COMM_CD_y":"level_1"})
cr2["commodity"]= [row.replace(' ',',')for row in cr2["commodity"]]
col4=["class","commodity"]
cr2["combine"]= cr2[col4].apply(lambda row: ', '.join(row.values.astype(str)),axis=1)


In [4]:

df1= cr2[['combine','level_1']]
df2 = df1.append(unspec2,ignore_index=True,sort=False)


In [5]:
df2.head(5)

,combine,level_1
0,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",3510
1,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",3515
2,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",3520
3,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",3525
4,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",3526


In [6]:
df9 = df2.apply(lambda x: ','.join(x.astype(str)),axis = 1)

In [7]:
df9.head()

0    AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...
1    AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...
2    AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...
3    AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...
4    AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...
dtype: object

In [8]:
df_clean2 = pd.DataFrame({'clean':df9})

In [9]:
df_clean2.head()

,clean
0,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU..."
1,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU..."
2,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU..."
3,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU..."
4,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU..."


In [10]:
sent23 = [row.split(',') for row in df_clean2['clean']]


In [11]:
print(sent23[10542:10555])

[['Fresh cut amaranthuses', 'Fresh cut blooms of amaranthuses', ' Fresh cut upright green amaranthus', 'Fresh cut bloom of the upright green variety of amaranthus', '10311804'], ['Fresh cut amaranthuses', 'Fresh cut blooms of amaranthuses', ' Fresh cut upright red amaranthus', 'Fresh cut bloom of the upright red variety of amaranthus', '10311805'], ['Fresh cut amaryllises', 'Fresh cut blooms of amaryllises', ' ', '', 'nan'], ['Fresh cut amaryllises', 'Fresh cut blooms of amaryllises', ' Fresh cut naranja amaryllis', 'Fresh cut bloom of the naranja variety of amaryllises', '10311901'], ['Fresh cut amaryllises', 'Fresh cut blooms of amaryllises', ' Fresh cut orange nagano amaryllis', 'Fresh cut bloom of the orange nagano variety of amaryllises', '10311902'], ['Fresh cut amaryllises', 'Fresh cut blooms of amaryllises', ' Fresh cut pygmee mini amaryllis', 'Fresh cut bloom of the pygmee mini variety of amaryllises', '10311903'], ['Fresh cut amaryllises', 'Fresh cut blooms of amaryllises', '

In [12]:
print(len(sent23))

163115


In [13]:
model2 = gensim.models.word2vec.Word2Vec(sent23,min_count=1,size=300, workers=6, window= 2)

In [20]:
def NumberofDocs(li,nums):
    shortlist=[]
    number=0.0
    for i in li:
        if(i!='nan'):
            point = model2.wv.similarity(str(i),nums)
            if(point>0.8):
                shortlist.append(i) 
    
    return shortlist

result =NumberofDocs(nums,'4045') 
print(len(result))
print(result)
print(df2.loc[df2['level_1'].isin(result)])

47685
                                                  combine   level_1
6656    Livestock,, Alpaca,The alpaca -from quechua al...  10101518
6657    Livestock,, Buffalo or bison,Large, shaggy her...  10101519
6661                         Birds and fowl,, Live ducks,  10101602
6666    Birds and fowl,, Live guinea fowl,Type of bird...  10101607
6672                               Live fish,, Live eels,  10101705
6674    Live fish,, Live palometa fish or mylossoma au...  10101707
6676    Live fish,, Live red belly pacu fish,A type of...  10101709
6678    Live fish,, Live paiche fish,A type of tropica...  10101711
6681    Live fish,, Live maparate fish,A type of catfi...  10101714
6682    Live fish,, Live lumptail sea robin fish,A typ...  10101715
6684    Live fish,, Live ispi fish,A type of pupfish n...  10101717
6685    Live fish,, Live frigate tuna fish or melva fi...  10101718
6687    Live fish,, Live acarahuazu fish,A tropical fr...  10101720
6689    Live fish,, Live armored catfish o

In [21]:
result2 =NumberofDocs(nums,'3526') 
print(len(result2))
print(df2.loc[df2['level_1'].isin(result2)])

351
                                                 combine   level_1
7012   Residues other than animal feed,, Residue of l...  10152103
17226       Earth moving machinery,, Combat earthmovers,  22101534
18005  Metal bending machines,Machines that apply dir...  23251504
18542  Securing and protecting supplies,, Tamper proo...  24141504
18918           Braking systems and components,, Rotors,  25171705
19158             Vehicle interior systems,, Headliners,  25174404
19536                       Engines,, Pneumatic engines,  26101502
19597  Engine components and accessories,, Carburetor...  26101756
19743                         Clutches,, Plate clutches,  26111901
19950  Exhaust structures or screening equipment,, St...  26131606
20109                           Forming tools,, Mallets,  27111601
20297  Agriculture, forestry and garden handtools,A s...  27112012
20534  Tool attachments and accessories,The comoditie...  27112820
20725                     Beams,, Stainless steel beams,  

In [16]:
model2.wv.similarity('10101707','4045')

0.83012986

In [17]:
model2.wv.similarity('85903723','4045')

0.6577544

In [18]:
model2.wv.most_similar('3105')[:15]

[(' (MILIARIA', 0.9131325483322144),
 (' WITHOUT COMA K71.10)', 0.9128967523574829),
 ('11-diol', 0.9126417636871338),
 ('This classification denotes the group of activities that yields a surgical intervention or procedure to removal of drainage device from retroperitoneum',
  0.9125745296478271),
 (' (OTH DISORDERS RESULTING FROM IMPAIRED RENAL TUBULAR FUNCTION N25.89)',
  0.9122788906097412),
 (' virus not identified', 0.9122744798660278),
 (' WITH COMA K71.11)', 0.9122116565704346),
 (' (OTHER VOMITING of NEWBORN P92.9)', 0.9121987819671631),
 (' (SEPSIS of NEWBORN due to other STREPTOCOCCI P36.19)', 0.9121073484420776),
 (' UNSPECIFIED L29.3)', 0.9119848012924194)]

In [19]:
"""
book = open_workbook('UNSPSC English v220601 project.xlsx')
'''book = open_workbook('Unspec List2b.xlsx')'''
'''To work on the UNSPSC sheet you need to change the values of 0 to 12 and 1 to
16 in order to make the it work.'''
dict_list = []
sheet = book.sheet_by_index(0)
# read header values into the list
keys = [sheet.cell(0, col_index).value for col_index in range(sheet.ncols)]

for row_index in range(1, sheet.nrows):
    d = {keys[col_index]: sheet.cell(row_index, col_index).value
         for col_index in range(sheet.ncols)}
    dict_list.append(d)

print(len(dict_list))
    
doclist =[]
listOfEntry =[]
entrylist = []
df7 = pd.DataFrame()


for entry in dict_list:
    if(entry.get("Family")==""):
        continue
    elif(entry.get("Family")!="" and entry.get("Class")==""):
        if(listOfEntry!=[]):
            #print(listOfEntry)
            doclist += entrylist
            continue
    elif(entry.get("Commodity")==""):
        continue
    else:
        if(entry.get("Class Definition")!=""):
            f = str(sent_tokenize(entry.get("Class Definition").lower())).strip('[]').strip('\'')
            #print(f)
        else:
            f=str([" "]).strip('[]').strip('\'')
        e = str(sent_tokenize(entry.get("Class Title").lower())).strip('[]').strip('\'')
        #print(e)
        g = str(sent_tokenize(entry.get("Commodity Title").lower())).strip('[]').strip('\'')
        #print(g)
        if(entry.get("Commodity Definition")!=""):
            h = str(sent_tokenize(entry.get("Commodity Definition").lower())).strip('[]').strip('\'')
            #print(h)
        else:
            h = str([" "]).strip('[]').strip('\'')
        i=int(entry.get("Commodity"))
        df7 = df7.append({'class':e+f,'commodity':g+h,'level_1': i},ignore_index=True)
        
  
        
doclist.append(listOfEntry) 

print("done")        




model2.wv.vocab
model2.save("W2V-Model")

text = "AIR CONDITIONING, HEATING, AND VENTILATING: EQUIPMENT, PARTS  HYDRONIC SPECIALTIES"
idt =['101'] 
text = text.lower()
newEntry = []
newEn =[]
stop_words = set(stopwords.words('english'))
ch = [ '``','``', "''",',','.','\\n',"'",";",":","(",")","-","--"]


Wtest = word_tokenize(text)
Wtest = [ j for j in Wtest if not j in ch ]
Wtest = [w for w in Wtest if not w in stop_words]
newEntry.append(Wtest)

def concatenate_list_data(list):
    result= ''
    for element in list:
        result += str(element+" ")
    return result

newEntry[0] = concatenate_list_data(newEntry[0])
newEntry += idt
newEn.append(newEntry)
print(newEn)

new_mod = gensim.models.Word2Vec.load("W2V-Model")
new_mod.build_vocab(newEn,update = True)
new_mod.train(newEn,total_examples=1,epochs = 1)

new_mod.wv.similarity('30121703.0','101')
new_mod.wv.most_similar_cosmul('101')

new_mod.wv.similar_by_vector('101')

"""

'\nbook = open_workbook(\'UNSPSC English v220601 project.xlsx\')\n\'\'\'book = open_workbook(\'Unspec List2b.xlsx\')\'\'\'\n\'\'\'To work on the UNSPSC sheet you need to change the values of 0 to 12 and 1 to\n16 in order to make the it work.\'\'\'\ndict_list = []\nsheet = book.sheet_by_index(0)\n# read header values into the list\nkeys = [sheet.cell(0, col_index).value for col_index in range(sheet.ncols)]\n\nfor row_index in range(1, sheet.nrows):\n    d = {keys[col_index]: sheet.cell(row_index, col_index).value\n         for col_index in range(sheet.ncols)}\n    dict_list.append(d)\n\nprint(len(dict_list))\n    \ndoclist =[]\nlistOfEntry =[]\nentrylist = []\ndf7 = pd.DataFrame()\n\n\nfor entry in dict_list:\n    if(entry.get("Family")==""):\n        continue\n    elif(entry.get("Family")!="" and entry.get("Class")==""):\n        if(listOfEntry!=[]):\n            #print(listOfEntry)\n            doclist += entrylist\n            continue\n    elif(entry.get("Commodity")==""):\n        